<a href="https://colab.research.google.com/github/hishaam19/RockID/blob/TrainingModelsAlex/jim_efficientnet_gemstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.11.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 91.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 99.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 107.1 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.0
    Uninstalling tensorboard-data-server-0.7.0:
      Successfully uninstalled tensorboard-data-serv

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import layers
import tensorflow as tf
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.optimizers import Adam

In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
img_width=380
img_height=380
batch_size=140
num_of_classes=85

training_set_directory = "/content/drive/MyDrive/Datasets/gemstone_roboflow_2/train"
validation_set_directory = "/content/drive/MyDrive/Datasets/gemstone_roboflow_2/valid"

In [ ]:
training_data = ImageDataGenerator( ##rescale = 1./255,
                                   horizontal_flip=True,
                                    vertical_flip = True,
                                   )

validation_data = ImageDataGenerator ( ##rescale = 1./255,
                                      horizontal_flip=True,
                                      vertical_flip = True,
                                    )


In [ ]:
training_gen = training_data.flow_from_directory(training_set_directory,
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle = True,
                                                 target_size = (img_height, img_width))

Found 4948 images belonging to 85 classes.


In [ ]:
validation_gen = validation_data.flow_from_directory(validation_set_directory,
                                                     batch_size = batch_size,
                                                     class_mode = 'categorical',
                                                     shuffle = True,
                                                     target_size = (img_height, img_width))

Found 282 images belonging to 85 classes.


In [ ]:
efficientNetB4 = tf.keras.applications.EfficientNetB4(include_top = False, weights='imagenet', drop_connect_rate=0.4)

71686520/71686520 [==============================] - 3s 0us/step


In [ ]:
efficientNetB4.trainable = False


In [ ]:
model=Sequential()
model.add(efficientNetB4)
model.add(GlobalAveragePooling2D(name="avg_pool"))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(num_of_classes, activation = "softmax"))

In [ ]:
model.compile(
        optimizer=Adam(learning_rate=1e-2), loss="categorical_crossentropy", metrics=["accuracy"]
    )

In [ ]:
step_size_train = training_gen.n/training_gen.batch_size

In [ ]:
history = model.fit(training_gen,
                                epochs=50,
                                verbose=1,
                                steps_per_epoch = step_size_train,
                                validation_data=validation_gen
                                )

Epoch 1/50
35/35 [==============================] - 2205s 62s/step - loss: 4.0965 - accuracy: 0.3436 - val_loss: 2.0103 - val_accuracy: 0.5177
Epoch 2/50
35/35 [==============================] - 73s 2s/step - loss: 2.0532 - accuracy: 0.5790 - val_loss: 1.8476 - val_accuracy: 0.4787
Epoch 3/50
35/35 [==============================] - 73s 2s/step - loss: 1.5753 - accuracy: 0.6419 - val_loss: 1.6861 - val_accuracy: 0.5496
Epoch 4/50
35/35 [==============================] - 72s 2s/step - loss: 1.2796 - accuracy: 0.6880 - val_loss: 1.6597 - val_accuracy: 0.5319
Epoch 5/50
35/35 [==============================] - 73s 2s/step - loss: 0.9342 - accuracy: 0.7512 - val_loss: 1.5283 - val_accuracy: 0.5993
Epoch 6/50
35/35 [==============================] - 73s 2s/step - loss: 0.9613 - accuracy: 0.7363 - val_loss: 1.6268 - val_accuracy: 0.5638
Epoch 7/50
35/35 [==============================] - 73s 2s/step - loss: 0.8288 - accuracy: 0.7751 - val_loss: 1.6430 - val_accuracy: 0.5709
Epoch 8/50
35/35 

In [ ]:
print(tf.__version__)

2.11.0


In [ ]:
for layer in efficientNetB4.layers[-20:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

In [ ]:
for layer in model.layers[-20:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

In [ ]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss="categorical_crossentropy", metrics=["accuracy"])


In [ ]:
history = model.fit(training_gen,
                                epochs=50,
                                verbose=1,
                                steps_per_epoch = step_size_train,
                                validation_data=validation_gen
                                )

NameError: ignored